In [1]:
import os
from os import walk

import pandas as pd
import numpy as np
import sys
import re

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from collections import Counter

%matplotlib inline

pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 1500)
pd.set_option('max_colwidth', 400)

In [2]:
#Set to False if no subsampling, else set to the desired number for subsample results:
#subsampling = "1"
subsampling = False

#For a special case of looking only at samples with 200X coverage:
#high_coverage_analysis = False
high_coverage_analysis = False

#Failsafe:
if high_coverage_analysis == True:
    subsampling = False

In [3]:
main_folder_path = 'C:\\Users\\nikol\\OneDrive\\DTU\\11_semester\\'

resultpath = main_folder_path + 'output_full_dataset\\'

#Outcomment all but the resolution desired:
if subsampling != False:
    resultpath = main_folder_path + 'output_' + subsampling + 'X\\' + '1000_genomes_results\\'

gold_standard_path = main_folder_path + 'gold_standard_data\\'

In [4]:
p_group_filepath = gold_standard_path + 'p_group_resolution.txt'
g_group_filepath = gold_standard_path + 'g_group_resolution.txt'

In [5]:
#Chose allele resolution: options: "one_field", two_field", "p_group" or "e_group"
resolution = "two_field"

### Function for flattening a list

In [6]:
def flatten(a):
    return [item for sublist in a for item in sublist if item != []]

# Functions for Allele conversion

In [7]:
#Function for converting an allele to one/two/three field resolution (disregarding any trailing letters - still unambiguous)
#For the genes A, B, C, DRB1 and DQB1 the first field always has two digits.

def convert_to_one_field(allele_high_res):
    one_field_finder = re.search(r"(A|B|C|DRB1|DQB1)\*\d{2}", allele_high_res)
    
    if one_field_finder != None:
        one_field_finder = one_field_finder.group(0)
    else:
        one_field_finder = None
    
    return one_field_finder    

    
def convert_to_two_field(allele_high_res):
    two_field_finder = re.search(r"(A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}", allele_high_res)
    
    if two_field_finder != None:
        allele_two_field = two_field_finder.group(0)
    else:
        allele_two_field = None
    
    return allele_two_field    


def convert_to_three_field(allele_high_res):
    six_field_finder = re.search(r"(A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}:\d{2,3}", allele_high_res)
    
    if six_field_finder != None:
        allele_six_field = six_field_finder.group(0)
    else:
        allele_six_field = None
    
    return allele_six_field    

## Function for P-group conversion

In [8]:
#Make dict for P-type conversion

p_group_filepath = gold_standard_path + 'p_group_resolution.txt'

#Dict with the structure: {allele_in : allele_converted_to_p_group...}
p_group_dict = dict()

#List all null alleles not part of a G group 
null_allele_outside_g_group_list = list()


#Read the important results:
with open(p_group_filepath, 'r') as infile:
    for line in infile:
        
        #If allele doesn't belong to a P group, add it to dict as key and value
        if ('/' not in line) and (line[0] != '#'):
            gene = line.split('*')[0]
            
            #Only register the valid alleles:
            if gene in ['A', 'B', 'C', 'DRB1', 'DQB1']:
                p_group_entry = convert_to_two_field(gene + "*" + line.split(';')[-2])
                
                p_group_dict[p_group_entry] = p_group_entry
                
        
        #If several alleles map to the same one, they are separated by a "/" - this indicates a P group
        if ('/' in line) and (line[0] != '#'):
            gene = line.split('*')[0]
            
            #Only register the valid alleles:
            if gene in ['A', 'B', 'C', 'DRB1', 'DQB1']:
                        
                #Find the four field, G type resolution. The G type is found at the end of the line.
                p_group_full = gene + "*" + line.split(';')[-1][:-1]
                p_group_two_field = convert_to_two_field(p_group_full)
            
                #Read the rest of the alleles and clean up the front and end part
                synonymous_alleles = line.split('/')
                synonymous_alleles[0] = synonymous_alleles[0].split(';')[1]
                synonymous_alleles[-1] = synonymous_alleles[-1].split(';')[0]
                

                #Convert all alleles to four field resolution
                for i in range(len(synonymous_alleles)):
                    synonymous_alleles[i] = gene + "*" + synonymous_alleles[i]
                    synonymous_alleles[i] = convert_to_two_field(synonymous_alleles[i])

                #Remove duplicates when converting to four field:
                synonymous_allels_unique_two_field = list(set(synonymous_alleles))
                
                #Add key in dict for each of the unique entries:
                for synonymous_allele in synonymous_allels_unique_two_field:
                    
#                     #Check for ambiguities in the G-type conversion e.g. C*02:02:02G and C*02:10:01G have same exon sequence for exon 2 and 3
#                     if synonymous_allele in p_group_dict.keys():
#                         print(synonymous_allele, p_group_full)
                    
                    p_group_dict[synonymous_allele] = convert_to_two_field(p_group_full)
                                    

#Finally, add the alleles, which have been changed/updated and are included in the gold standardd/predictions manually
#e.g. A*01:34N, "Sequence shown to be expressed at low levels and renamed A*01:01:38L (March 2011)" (https://www.ebi.ac.uk/cgi-bin/ipd/imgt/hla/allele.cgi)               
p_group_dict['A*01:34'] = 'A*01:01'    
p_group_dict['C*03:99'] = 'C*01:169'
p_group_dict['A*03:260'] = 'A*03:284' 
    
    
    
#Make P type conversion function using p_group_dict
def convert_to_p_group(allele):

    #Start by converting to two field:
    allele_two_field = convert_to_two_field(allele)
    
    #Find corresponding P-type if it exists. 
    if allele_two_field in p_group_dict:
        allele_two_field_p_group = p_group_dict[allele_two_field]
    
    #Failsafe: if allele isn't found in p_group_dict (which it should be), then just use two_field_resolution
    else:
        allele_two_field_p_group = allele_two_field
        #print("Allele was not found in p_group_dict", allele_two_field)
       
        
    return allele_two_field_p_group

# NetMHCseq typing resolution
Using Evaxion typing (e_group)

In [9]:
#Make dict for evaxion-group conversion

e_group_filepath = gold_standard_path + 'e_group_resolution.txt'

e_group_dict = dict()


#Make conversion from full allele names to G groups
with open(e_group_filepath, 'r') as infile:
    
    #Add all relevant entries from e_group_filepath to e_group_dict
    for line in infile:
        if line.startswith('HLA-'):
            gene = line[4]
            
            if gene in ['A', 'B', 'C']:     
                 
                digits_from_type = re.search(r'\d{2}:\d{2,3}',line)
                
                if digits_from_type != None:
                    allele = gene + "*" + digits_from_type.group(0)

                    peptide_sequence = line.split('\t')[-1][:-1]

                    e_group_dict[allele] = peptide_sequence

            
        elif line.startswith('DRB1'):
            gene = 'DRB1'
            
            digits_from_type_raw = line.split('_')[1].split('\t')[0]
            
            digits_from_type = digits_from_type_raw[0:2] + ':' + digits_from_type_raw[2:]
            
            allele = gene + "*" + digits_from_type
                
            peptide_sequence = line.split('\t')[-1][:-1]
            
            e_group_dict[allele] = peptide_sequence

            

#Function for converting to e-group resolution:
def convert_to_e_group(allele):

    #Start by converting to P group:
    allele_p_group = convert_to_p_group(allele)
    
    #Find corresponding e-type if it exists
    if allele_p_group in e_group_dict:
        allele_e_group = e_group_dict[allele_p_group]
    
    #Else use P group
    else:
        allele_e_group = allele_p_group
        
    return allele_e_group



## Create one conversion function (1-field, 2-field, P group, NetMHCseq group)

In [10]:
#Make function for conversion, to be used in this run of the notebook:

def convert_allele(allele, conversion_type = resolution):

    """
    input:
    allele (str): An allele in the format (A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}:?\d{0,3}G?:?\d{0,3} to be converted.
    resolution:   the resolution, with which the allele is converted to

    """
    if conversion_type == "one_field":
        converted_allele = convert_to_one_field(allele)
    
    elif conversion_type == "two_field":
        converted_allele = convert_to_two_field(allele)
        
    elif conversion_type == "p_group":
        converted_allele = convert_to_p_group(allele)
    
    elif conversion_type == "e_group":
        converted_allele = convert_to_e_group(allele)
        
    else:
        print("A conversion mistake happend. Please specify a correct conversion type.")
        converted_allele = None
        
    return converted_allele      

# Load Typing Results by HLA Typing tools:

## Kourami
Save both single and ambiguous results

In [11]:
kourami_result_filepath = resultpath + 'kourami\\'

kourami_log_filepath = resultpath + 'kourami_results_from_logfiles\\'

kourami_files = list()
kourami_logfiles = list()

#Initalize result dict for single guess and for multiple typing
kourami_results = dict()
kourami_results_ambiguous = dict()


for (dirpath, dirnames, filenames) in walk(kourami_result_filepath):
    kourami_files.extend(filenames)
    
for (dirpath, dirnames, filenames) in walk(kourami_log_filepath):
    kourami_logfiles.extend(filenames)
    

for filename in kourami_files:
    #Don't include the performance logs
    if filename.endswith('.txt'):
        
        #If file is not empty:
        if os.stat(kourami_result_filepath + filename).st_size != 0:
            temp_result_dict = dict()
            temp_result_dict_ambiguous = dict()
    
            with open(kourami_result_filepath + filename, 'r') as infile:

                for line in infile:
                    #Find the first match / allele prediction
                    allele_searcher = re.search(r"(A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}:?\d{0,3}G?:?\d{0,3}", line)
                    
                    #Find all allele predictions (ambiguous). ?: is required for non-capturing groups
                    allele_searcher_ambiguous = re.findall(r"(?:A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}:?\d{0,3}G?:?\d{0,3}", line)
                    
                    if allele_searcher is not None:                        
                        found_allele = allele_searcher.group(0)
                        
                        gene = re.search(r"(A|B|C|DRB1|DQB1)", found_allele).group(0)
                        
                        #Convert alleles to right resolution:
                        pred_converted = convert_allele(found_allele)
                        
                        allele_searcher_ambiguous_converted = [convert_allele(i) for i in allele_searcher_ambiguous]
                        
                        if (resolution ==  "two_field") and (pred_converted in null_allele_outside_g_group_list):
                            print(filename[:-4], pred_converted, sep = "\t")
                        
                        
                        #Add to list of predictions for this sample
                        if gene not in temp_result_dict.keys():
                            temp_result_dict[gene] = [[pred_converted]]
                            temp_result_dict_ambiguous[gene] = [allele_searcher_ambiguous_converted]
                        else:
                            temp_result_dict[gene] += [[pred_converted]]
                            temp_result_dict_ambiguous[gene] += [allele_searcher_ambiguous_converted]

            #Add sample prediction to dict
            kourami_results[filename[:-4]] = temp_result_dict
            kourami_results_ambiguous[filename[:-4]] = temp_result_dict_ambiguous
            
            
        #If file is empty, add an empty dict.
        else:
            kourami_results[filename[:-4]] = dict()
            kourami_results_ambiguous[filename[:-4]] = dict()
            #print("Unable to load results for sample" + filename[:-4] + ". No prediction added")                           
                
#kourami_results

## HLA-LA

In [12]:
hla_la_result_filepath = resultpath + 'hla-la\\'

hla_la_files = []
for (dirpath, dirnames, filenames) in walk(hla_la_result_filepath):
    hla_la_files.extend(filenames)
    
#print(hla_la_files)


hla_la_results = dict()

for filename in hla_la_files:
    if filename.endswith('.txt'):
        temp_results_object = pd.read_csv(hla_la_result_filepath + filename, sep = "\t")['Allele']
        temp_results = [i for i in temp_results_object if i.startswith(('A', 'B', 'C', 'DRB1', 'DQB1'))]
        
        
        #Make dict of dicts for results:
        temp_result_dict = dict()
        
        for pred in temp_results:
            gene = re.search(r"(A|B|C|DRB1|DQB1)", pred).group(0)
            
            pred_converted = convert_allele(pred)
            
            if (resolution ==  "two_field") and (pred_converted in null_allele_outside_g_group_list):
                        print(filename[:-4], pred_converted, sep = "\t")

            
            
            #Add to list of predictions for this sample
            if gene not in temp_result_dict.keys():
                temp_result_dict[gene] = [[pred_converted]]
            else:
                temp_result_dict[gene] += [[pred_converted]]

        
        hla_la_results[filename[:-4]] = temp_result_dict


#hla_la_results

## Optitype

In [13]:
optitype_result_filepath = resultpath + 'optitype\\'

optitype_files = []
for (dirpath, dirnames, filenames) in walk(optitype_result_filepath):
    optitype_files.extend(filenames)
    
#print(optitype_files)


optitype_results = dict()

for filename in optitype_files:
    
    #Check for right file, and that the file is not empty
    if (filename.endswith('.txt')) and (os.stat(optitype_result_filepath + filename).st_size != 0):
        temp_results_raw = list(pd.read_csv(optitype_result_filepath + filename, sep = "\t").iloc[0])[1:7]

        temp_results = [i for i in temp_results_raw if isinstance(i,str)]

        
        
        #Make dict of dicts for results:
        temp_result_dict = dict()
        
        for pred in temp_results:
            gene = re.search(r"(A|B|C|DRB1|DQB1)", pred).group(0)
            
            pred_converted = convert_allele(pred)
            
            if (resolution ==  "two_field") and (pred_converted in null_allele_outside_g_group_list):
                        print(filename[:-4], pred_converted, sep = "\t")            
            
            
            #Add to list of predictions for this sample
            if gene not in temp_result_dict.keys():
                temp_result_dict[gene] = [[pred_converted]]
            else:
                temp_result_dict[gene] += [[pred_converted]]
      
        optitype_results[filename[:-4]] = temp_result_dict
 
    #Add empty entry for empty file
    elif (filename.endswith('.txt')) and (os.stat(optitype_result_filepath + filename).st_size == 0):
        optitype_results[filename[:-4]] = dict()
        
#optitype_results

## Hisatgenotype

In [14]:
hisatgenotype_result_filepath = resultpath + 'hisatgenotype\\'

hisatgenotype_files = []
for (dirpath, dirnames, filenames) in walk(hisatgenotype_result_filepath):
    hisatgenotype_files.extend(filenames)
    
#print(hisatgenotype_files)

#Save two predictions. One, with one guess per allele and one with the full prediction
hisatgenotype_results = dict()

for filename in hisatgenotype_files:
    if filename.endswith('.txt'):
    
        hisatgenotype_resultlist = list()
        
        hisatgenotype_resultlist_ambiguous = list()
      
        with open(hisatgenotype_result_filepath + filename) as infile:
            for line in infile:
                result = re.match(r'^\t+(1|2)\sranked (A|B|C|DRB1|DQB1)',line)
            
                if result is not None:
                    hisatgenotype_resultlist.append(line.split()[2])              
                     
            #Duplicate prediction for an allele in case of homologous case, so that each gene has two predictions.
            #In a homologous case, both result dicts only have one prediction and both needs an update.
            for allele in ['A', 'B', 'C', 'DRB1', 'DQB1']:
                allele_list = [pred for pred in hisatgenotype_resultlist if pred.startswith(allele)]

                if len(allele_list) == 1:
                    hisatgenotype_resultlist.append(allele_list[0])
                    hisatgenotype_resultlist.sort()

            temp_results = hisatgenotype_resultlist
            
        #Make dict of dicts for results:
        temp_result_dict = dict()
        
        for pred in temp_results:
            gene = re.search(r"(A|B|C|DRB1|DQB1)", pred).group(0)
            
            pred_converted = convert_allele(pred)

            if (resolution ==  "two_field") and (pred_converted in null_allele_outside_g_group_list):
                        print(filename[:-4], pred_converted, sep = "\t")
            
            #Add to list of predictions for this sample
            if gene not in temp_result_dict.keys():
                temp_result_dict[gene] = [[pred_converted]]
            else:
                temp_result_dict[gene] += [[pred_converted]]


        hisatgenotype_results[filename[:-4]] = temp_result_dict
                
#hisatgenotype_results           
        

## STC-seq

In [15]:
stc_seq_result_filepath = resultpath + 'stc-seq\\'

stc_seq_files = []
for (dirpath, dirnames, filenames) in walk(stc_seq_result_filepath):
    stc_seq_files.extend(filenames)
    
#print(stc_seq_files)

stc_seq_results = dict()

for filename in stc_seq_files:
    if filename.endswith('.txt'):
        
        #Get raw results as lists
        stc_df = pd.read_csv(stc_seq_result_filepath + filename, sep = "\t").reset_index()
        stc_df.columns = stc_df.iloc[0]
        stc_list = list(stc_df[stc_df['Locus'].isin(['A', 'B', 'C','DRB1', 'DQB1'])]['Genotype'])
        stc_seq_alt_list = list(stc_df[stc_df['Locus'].isin(['A', 'B', 'C','DRB1', 'DQB1'])]['Alternative_genotype'].dropna())

        #Format results for single prediction
        temp_results =  flatten([i.split(',') for i in stc_list])
        
        
        #Make dict of dicts for results:
        temp_result_dict = dict()
        
        for pred in temp_results:
            gene = re.search(r"(A|B|C|DRB1|DQB1)", pred).group(0)

            pred_converted = convert_allele(pred)

            if (resolution ==  "two_field") and (pred_converted in null_allele_outside_g_group_list):
                        print(filename[:-4], pred_converted, sep = "\t")
            
            #Add to list of predictions for this sample
            if gene not in temp_result_dict.keys():
                temp_result_dict[gene] = [[pred_converted]]
            else:
                temp_result_dict[gene] += [[pred_converted]]

                
        #Add predictions to dicts
        stc_seq_results[filename[:-4]] = temp_result_dict

#stc_seq_results

## Depth analysis

In [16]:
#load data
depth_filepath = resultpath + 'depth\\'

depth_files = []
for (dirpath, dirnames, filenames) in walk(depth_filepath):

    depth_files.extend(filenames)

depth_results = dict()

for filename in depth_files:
    
    if filename.endswith('.depth.mosdepth.summary.txt'):
        temp_results = pd.read_csv(depth_filepath + filename, sep='\t')
        
        temp_results.set_index('chrom', inplace=True)
        
        mean_region_depth = temp_results.loc['total_region', 'mean']
        depth_results[filename[0:7]] = mean_region_depth
    
#depth_results

# Load Gold standard data

In [17]:
MG_exome_merged_df = pd.read_pickle(gold_standard_path + "MG_exome_merged_df.pkl")

gs_one_field_df = pd.read_pickle(gold_standard_path + "gs_one_field_df.pkl")

gs_two_field_df = pd.read_pickle(gold_standard_path + "gs_two_field_df.pkl")

gs_p_group_df = pd.read_pickle(gold_standard_path + "gs_p_group_df.pkl")

gs_e_group_df = pd.read_pickle(gold_standard_path + "gs_e_group_df.pkl")

gold_standard_id_list = list(MG_exome_merged_df.index)

In [18]:
#Only pick the 221 chosen samples, if this is for downsampling:
if (subsampling != False) and (high_coverage_analysis == False):
    gold_standard_id_list = list(kourami_results.keys())
    
#Look at 200X samples - list found in "Crate_downsample_subset_and_multipliers.ipynb" - this is 14 samples
if high_coverage_analysis == True:
    gold_standard_id_list = ['HG00731', 'HG00732', 'HG01756', 'HG01757', 'HG01872', 'HG01873', 'HG01886', 'HG01953', 'HG01968','HG02014', 'HG02057', 'NA18912', 'NA19648', 'NA20313']
        
    
MG_exome_merged_df = MG_exome_merged_df.loc[gold_standard_id_list]

gs_one_field_df = gs_one_field_df.loc[gold_standard_id_list]

gs_two_field_df = gs_two_field_df.loc[gold_standard_id_list]

gs_p_group_df = gs_p_group_df.loc[gold_standard_id_list]

gs_e_group_df = gs_e_group_df.loc[gold_standard_id_list] 

In [19]:
# Chose which dataframe should be the gold standard (pick between resolutions)
if resolution == "one_field":
    gold_standard_df = gs_one_field_df

elif resolution == "two_field":
    gold_standard_df = gs_two_field_df

elif resolution == "p_group":
    gold_standard_df = gs_p_group_df
    
elif resolution == "e_group":
    gold_standard_df = gs_e_group_df

In [20]:
#Load metadata:
gs_2018_filepath = gold_standard_path + '2018_1129_HLA_types_full_1000_Genomes_Project_panel.txt'

gs_2018_raw_df = pd.read_csv(gs_2018_filepath, sep = "\t")

#Rename columns
gs_2018_raw_df.rename(columns={'Sample ID': 'id', 'Population': 'sbgroup', 'HLA-A 1': 'A', 'HLA-A 2': 'A.1', 'HLA-B 1': 'B',
                            'HLA-B 2': 'B.1', 'HLA-C 1': 'C', 'HLA-C 2': 'C.1', 'HLA-DQB1 1': 'DQB1', 'HLA-DQB1 2': 'DQB1.1',
                            'HLA-DRB1 1': 'DRB1' , 'HLA-DRB1 2': 'DRB1.1' }, inplace=True)

#Set id as index
gs_2018_raw_df.set_index('id', inplace=True)

# Majority Votes

In [21]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

###  All votes (Kourami, HLA*LA, HISAT-Genotype and Optitype)

STC-Seq disregarded as it isn't strictly disigned for WES data

In [22]:
def get_prediction(result_dict, subject, allele):
    try:
        result = result_dict[subject][allele]
    except KeyError as error:
        result = ''
        
    return result

In [23]:
all_five_results = dict()

for subject in gold_standard_id_list:
    
    all_five_results[subject] = dict()

    for allele in ['A', 'B', 'C', 'DRB1', 'DQB1']:
        #Get list of all predictions:
        kourami_pred = get_prediction(kourami_results, subject, allele)
        hisatgenotype_pred = get_prediction(hisatgenotype_results, subject, allele)
        hla_la_pred = get_prediction(hla_la_results, subject, allele)
        optitype_pred = get_prediction(optitype_results, subject, allele)
        
        
        all_predictions_list = flatten([kourami_pred, hisatgenotype_pred, hla_la_pred, optitype_pred])
        
        #Remove empty results, if a tool was unable to return a prediction
        all_predictions_list_clean = flatten([preds for preds in all_predictions_list if (preds != []) and (preds != '')])
        
        #Count the amount of predictions for each allele (preds_count is a Counter object {allele : counts} )
        preds_count = Counter(all_predictions_list_clean)
        
        #Find the majority vote
        most_frequent_count = preds_count.most_common(2)[0][1]
        
        mode_1 = preds_count.most_common(2)[0][0]
        
        #Check, if any disagreement exists - if more than one allele was predicted. Else, predict homozygous.:
        if len(preds_count) == 1:
            mode_2 = mode_1
        
        #Investigate, if the second most frequent allele is frequent enought to consider heterozygosity
        else:
            
            second_most_frequent_count = preds_count.most_common(2)[1][1]
            
            #If the second most frequent prediction has more than half the predictions of the most predicted - assume heterozygous
            if second_most_frequent_count > most_frequent_count / 2: 
                mode_2 = preds_count.most_common(2)[1][0]
            
            #Else predict homozygous
            else:
                mode_2 = mode_1

                
        all_five_results[subject][allele] = [[mode_1], [mode_2]]    

### HLA-LA, Kourami, Hisatgenotype

In [24]:
graph_tools_results = dict()

for subject in gold_standard_id_list:
    
    graph_tools_results[subject] = dict()

    for allele in ['A', 'B', 'C', 'DRB1', 'DQB1']:
        #Get list of all predictions:
        kourami_pred = get_prediction(kourami_results, subject, allele)
        hisatgenotype_pred = get_prediction(hisatgenotype_results, subject, allele)
        hla_la_pred = get_prediction(hla_la_results, subject, allele)
        optitype_pred = get_prediction(optitype_results, subject, allele)
        stc_seq_pred = get_prediction(stc_seq_results, subject, allele)
        
        all_predictions_list = flatten([kourami_pred, hla_la_pred, hisatgenotype_pred])
        
        #Remove empty results, if a tool was unable to return a prediction
        all_predictions_list_clean = flatten([preds for preds in all_predictions_list if (preds != []) and (preds != '')])
        
        #Count the amount of predictions for each allele (preds_count is a Counter object {allele : counts} )
        preds_count = Counter(all_predictions_list_clean)
        
        #Find the majority vote
        most_frequent_count = preds_count.most_common(2)[0][1]
        
        mode_1 = preds_count.most_common(2)[0][0]
        
        #Check, if any disagreement exists - if more than one allele was predicted. Else, predict homozygous.:
        if len(preds_count) == 1:
            mode_2 = mode_1
        
        #Investigate, if the second most frequent allele is frequent enought to consider heterozygosity
        else:
            
            second_most_frequent_count = preds_count.most_common(2)[1][1]
            
            #If the second most frequent prediction has more than half the predictions of the most predicted - assume heterozygous
            if second_most_frequent_count > most_frequent_count / 2: 
                mode_2 = preds_count.most_common(2)[1][0]
            
            #Else predict homozygous
            else:
                mode_2 = mode_1
        
        graph_tools_results[subject][allele] = [[mode_1], [mode_2]]    
    

# Calculate Results - Typing Accuracy + Call Rate

In [25]:
def check_result(correct_alleles_list, tool_result_dict, subject_key, tool_name, allele):
     #There are two possible combinations between predictions and correct alleles.
    #1) pred[0] is a prediction for correct[0] and pred[1] for correct[1]
    #2) pred[1] is a prediction for correct[0] and pred[0] for correct[1]
    #I here pick the one of the two options, which gives the most correct prediction.
    
    try:
        correct_0_pred_0 = list(set(correct_alleles_list[0]).intersection(tool_result_dict[subject][allele][0]))
        correct_1_pred_1  = list(set(correct_alleles_list[1]).intersection(tool_result_dict[subject][allele][1]))

        option_1 = [correct_0_pred_0, correct_1_pred_1]

        correct_0_pred_1  = list(set(correct_alleles_list[0]).intersection(tool_result_dict[subject][allele][1]))
        correct_1_pred_0  = list(set(correct_alleles_list[1]).intersection(tool_result_dict[subject][allele][0]))

        option_2 = [correct_0_pred_1, correct_1_pred_0]

        hits_1 = len([i for i in option_1 if i != []])
        hits_2 = len([i for i in option_2 if i != []])

        num_correct_hits = max(hits_1,hits_2)
        
        
    except KeyError as error:
        num_correct_hits = 0
       
    #Register mistakes, if tool didn't get both alleles correct
    if num_correct_hits != 2:
        #Only register mistakes for optitype for HLA-A, HLA-B and HLA-C
        if allele in ['A', 'B', 'C']:
            error_dataframe_hla_I.loc[subject_key, tool_name]  += 2 - num_correct_hits
            
#             if tool_name != 'STC-seq':
#                 print(subject_key)
#                 print(tool_name, tool_result_dict[subject_key])
#                 print(correct_alleles_list)
#                 print(num_correct_hits)
#                 print("\n")
        
        #Avoid giving optitype errors for not typing HLA-II alleles
        elif tool_name == 'Optitype':
            pass
        
        else:
            error_dataframe_hla_II.loc[subject_key, tool_name]  += 2 - num_correct_hits
        
#             if tool_name != 'STC-seq':
#                 print(subject_key)
#                 print(tool_name, tool_result_dict[subject_key])
#                 print(correct_alleles_list)
#                 print(num_correct_hits)
#                 print("\n")
        
        
    return num_correct_hits



#Get the number of predictions from a tool for a specific allele for a specific subject (2 or 0)
def get_count(pred_dict, subject, allele):
    try:
        output = len([pred for pred in pred_dict[subject][allele]])
    except KeyError as error:
        output = 0
        
    return output

In [26]:
idx = pd.IndexSlice

#Create dataframe over samples which the tools didn't predict right. Samples as index

if subsampling == False:
    error_dataframe_hla_I = pd.DataFrame({'mean_depth' : depth_results})
    error_dataframe_hla_II = pd.DataFrame({'mean_depth' : depth_results})
else:
    error_dataframe_hla_I = pd.DataFrame({'dummy': kourami_results}).drop('dummy', axis = 1)
    error_dataframe_hla_II = pd.DataFrame({'dummy': kourami_results}).drop('dummy', axis = 1)

for tool in ['Kourami', 'HLA-LA', 'Optitype', 'Hisatgenotype', 'STC-seq', 'Kourami_ambiguous', 'ensemble_all', 'ensemble_graph']:
    error_dataframe_hla_I[tool] = 0

for tool in ['Kourami', 'HLA-LA', 'Hisatgenotype', 'STC-seq', 'Kourami_ambiguous', 'ensemble_all', 'ensemble_graph']:
    error_dataframe_hla_II[tool] = 0    

    
#Create score dataframe:
tool_extended_list = ['Kourami', 'Kourami', 'Kourami', 'Kourami', 'HLA-LA', 'HLA-LA',  'HLA-LA', 'HLA-LA', 'Optitype', 'Optitype', 'Optitype', 'Optitype', 'Hisatgenotype','Hisatgenotype', 'Hisatgenotype','Hisatgenotype', 'STC-seq', 'STC-seq', 'STC-seq', 'STC-seq', 'Kourami_ambiguous', 'Kourami_ambiguous', 'Kourami_ambiguous', 'Kourami_ambiguous', 'ensemble_all', 'ensemble_all', 'ensemble_all', 'ensemble_all', 'ensemble_graph', 'ensemble_graph', 'ensemble_graph', 'ensemble_graph', 'Total', 'Total', 'Total']
measure_list = ['score', 'count', 'call_rate', 'accuracy', 'score', 'count', 'call_rate', 'accuracy','score', 'count', 'call_rate', 'accuracy','score', 'count', 'call_rate', 'accuracy','score', 'count', 'call_rate', 'accuracy','score', 'count', 'call_rate', 'accuracy','score', 'count', 'call_rate', 'accuracy', 'score', 'count', 'call_rate', 'accuracy', 'count']

arrays = [tool_extended_list,measure_list]
tuples = list(zip(*arrays))

index = pd.MultiIndex.from_tuples(tuples, names=['Tool', 'Metric'])

#Make overall dataframe: (Evaxion is HLA-I + DRB1)
results_df = pd.DataFrame(0, index=['A', 'B', 'C', 'DRB1', 'DQB1', 'HLA-I', 'HLA-II', 'Evaxion', 'Total'], columns=index)

#Loop over subjects:
for subject in list(gold_standard_df.index):
    
    #Only include subjects which have been typed by the tools:
    if subject not in gold_standard_id_list:
        continue
      
    #Loop over alleles
    for allele in gold_standard_df.columns:
        

        results_df.loc[allele, idx['Total', 'count']] += 2
        
        facit = gold_standard_df.loc[subject,allele]

        
        results_df.loc[allele, idx['Kourami', 'score']]  += check_result(facit, kourami_results, subject, 'Kourami', allele)
        results_df.loc[allele, idx['Kourami', 'count']]  += get_count(pred_dict = kourami_results, subject = subject, allele = allele)
      
 
        results_df.loc[allele, idx['HLA-LA', 'score']]+= check_result(facit, hla_la_results, subject, 'HLA-LA', allele)
        results_df.loc[allele, idx['HLA-LA', 'count']]+= get_count(pred_dict = hla_la_results, subject = subject, allele = allele)

    
        results_df.loc[allele, idx['Optitype', 'score']] += check_result(facit, optitype_results, subject, 'Optitype', allele)
        results_df.loc[allele, idx['Optitype', 'count']]+= get_count(pred_dict = optitype_results, subject = subject, allele = allele)
        
        
        results_df.loc[allele, idx['Hisatgenotype', 'score']] += check_result(facit, hisatgenotype_results, subject, 'Hisatgenotype', allele)
        results_df.loc[allele, idx['Hisatgenotype', 'count']]+= get_count(pred_dict = hisatgenotype_results, subject = subject, allele = allele)
        
        
        results_df.loc[allele, idx['STC-seq', 'score']] += check_result(facit, stc_seq_results, subject, 'STC-seq', allele)
        results_df.loc[allele, idx['STC-seq', 'count']]+= get_count(pred_dict = stc_seq_results, subject = subject, allele = allele)
        
        #Kourami Ambiguous
        
        results_df.loc[allele, idx['Kourami_ambiguous', 'score']]  += check_result(facit, kourami_results_ambiguous, subject, 'Kourami_ambiguous', allele)
        results_df.loc[allele, idx['Kourami_ambiguous', 'count']]  += get_count(pred_dict = kourami_results_ambiguous, subject = subject, allele = allele)

        
        #The majority vote methods:
        
        results_df.loc[allele, idx['ensemble_all', 'score']] += check_result(facit, all_five_results, subject, 'ensemble_all', allele)
        results_df.loc[allele, idx['ensemble_all', 'count']]+= get_count(pred_dict = all_five_results, subject = subject, allele = allele)
        
        
        results_df.loc[allele, idx['ensemble_graph', 'score']] += check_result(facit, graph_tools_results, subject, 'ensemble_graph', allele)
        results_df.loc[allele, idx['ensemble_graph', 'count']]+= get_count(pred_dict = graph_tools_results, subject = subject, allele = allele)

In [27]:
gold_standard_df

,A,B,C,DRB1,DQB1
id,,,,,
NA06985,"[[A*03:01], [A*02:01]]","[[B*07:02], [B*57:01]]","[[C*07:02], [C*06:02]]","[[DRB1*15:01], [DRB1*15:01]]","[[DQB1*06:02], [DQB1*06:02]]"
NA06986,"[[A*03:01], [A*32:01]]","[[B*44:03], [B*44:03]]","[[C*04:01], [C*16:01]]","[[DRB1*07:01], [DRB1*07:01]]","[[DQB1*02:02], [DQB1*02:02]]"
NA06994,"[[A*02:01], [A*32:01]]","[[B*40:02], [B*08:01]]","[[C*02:02], [C*07:01, C*07:06]]","[[DRB1*01:01], [DRB1*04:04]]","[[DQB1*05:01], [DQB1*03:02]]"
NA07000,"[[A*02:01], [A*68:01]]","[[B*44:02], [B*40:01]]","[[C*03:03, C*03:04], [C*07:04]]","[[DRB1*03:01], [DRB1*11:01]]","[[DQB1*02:01], [DQB1*03:01]]"
NA07037,"[[A*30:01], [A*31:01]]","[[B*15:10], [B*40:01]]","[[C*03:04], [C*03:04]]","[[DRB1*04:04], [DRB1*13:02]]","[[DQB1*03:02], [DQB1*06:04]]"
NA07048,"[[A*02:01], [A*02:01]]","[[B*44:02], [B*07:02]]","[[C*05:01], [C*07:02]]","[[DRB1*04:01], [DRB1*15:01]]","[[DQB1*03:01], [DQB1*06:02]]"
NA07051,"[[A*02:01], [A*68:01]]","[[B*15:19, B*15:01, B*15:12], [B*07:02]]","[[C*03:03], [C*07:02]]","[[DRB1*04:01], [DRB1*15:01]]","[[DQB1*03:01], [DQB1*06:02]]"
NA07056,"[[A*01:01], [A*02:01]]","[[B*08:01], [B*57:01]]","[[C*07:01, C*07:06], [C*06:02]]","[[DRB1*03:01], [DRB1*07:01]]","[[DQB1*02:01], [DQB1*03:03]]"
NA07347,"[[A*30:02], [A*26:01]]","[[B*18:01], [B*44:02]]","[[C*05:01], [C*05:01]]","[[DRB1*03:01], [DRB1*11:04]]","[[DQB1*02:01], [DQB1*03:01]]"


In [28]:
#Calculate counts for Total:
#Add for Total:
results_df.loc['HLA-I', idx['Total', 'count']] += sum(results_df.loc[['A','B','C'], idx['Total', 'count']])
results_df.loc['HLA-II', idx['Total', 'count']] += sum(results_df.loc[['DRB1','DQB1'], idx['Total', 'count']])
results_df.loc['Evaxion', idx['Total', 'count']] += sum(results_df.loc[['HLA-I','DRB1'], idx['Total', 'count']])
results_df.loc['Total', idx['Total', 'count']] += sum(results_df.loc[['HLA-I','HLA-II'], idx['Total', 'count']])


#Loop over subjects:
for method in ['HLA-LA', 'Hisatgenotype', 'Kourami', 'Optitype', 'STC-seq', 'Kourami_ambiguous', 'ensemble_all', 'ensemble_graph']:
    
    #Calculate scores for groups of alleles:
    results_df.loc['HLA-I', idx[method, 'score']] += sum(results_df.loc[['A','B','C'], idx[method, 'score']])
    results_df.loc['HLA-II', idx[method, 'score']] += sum(results_df.loc[['DRB1','DQB1'], idx[method, 'score']])
    results_df.loc['Evaxion', idx[method, 'score']] += sum(results_df.loc[['HLA-I','DRB1'], idx[method, 'score']])
    results_df.loc['Total', idx[method, 'score']] += sum(results_df.loc[['HLA-I','HLA-II'], idx[method, 'score']])
         
    results_df.loc['HLA-I', idx[method, 'count']] += sum(results_df.loc[['A','B','C'], idx[method, 'count']])
    results_df.loc['HLA-II', idx[method, 'count']] += sum(results_df.loc[['DRB1','DQB1'], idx[method, 'count']])
    results_df.loc['Evaxion', idx[method, 'count']] += sum(results_df.loc[['HLA-I','DRB1'], idx[method, 'count']])
    results_df.loc['Total', idx[method, 'count']] += sum(results_df.loc[['HLA-I','HLA-II'], idx[method, 'count']])
                                                        
        
    for gene in list(results_df.index):

        results_df.loc[gene, idx[method, 'call_rate']] += round(results_df.loc[gene, idx[method, 'count']] / results_df.loc[gene, idx['Total', 'count']]*100,2)
        
        results_df.loc[gene, idx[method, 'accuracy']] += round(results_df.loc[gene, idx[method, 'score']] / results_df.loc[gene, idx['Total', 'count']]*100,2)
        
        

In [29]:
results_df

Tool    Kourami                            HLA-LA                           \
Metric    score   count call_rate accuracy  score count call_rate accuracy   
A        1420.0  1600.0     96.50    85.65   1538  1658     100.0    92.76   
B        1296.0  1432.0     86.37    78.17   1619  1658     100.0    97.65   
C        1361.0  1500.0     90.47    82.09   1603  1658     100.0    96.68   
DRB1     1591.0  1648.0     99.40    95.96   1616  1658     100.0    97.47   
DQB1     1226.0  1388.0     83.72    73.94   1559  1658     100.0    94.03   
HLA-I    4077.0  4532.0     91.11    81.97   4760  4974     100.0    95.70   
HLA-II   2817.0  3036.0     91.56    84.95   3175  3316     100.0    95.75   
Evaxion  5668.0  6180.0     93.18    85.46   6376  6632     100.0    96.14   
Total    6894.0  7568.0     91.29    83.16   7935  8290     100.0    95.72   

Tool    Optitype                            Hisatgenotype                    \
Metric     score   count call_rate accuracy         score   count call_rate   
A         1640.0  1658.0     100.0    98.91        1343.0  1658.0     100.0   
B         1623.0  1658.0     100.0    97.89        1541.0  1658.0     100.0   
C         1634.0  1658.0     100.0    98.55        1538.0  1658.0     100.0   
DRB1         0.0     0.0       0.0     0.00        1466.0  1658.0     100.0   
DQB1         0.0     0.0       0.0     0.00        1474.0  1658.0     100.0   
HLA-I     4897.0  4974.0     100.0    98.45        4422.0  4974.0     100.0   
HLA-II       0.0     0.0       0.0     0.00        2940.0  3316.0     100.0   
Evaxion   4897.0  4974.0      75.0    73.84        5888.0  6632.0     100.0   
Total     4897.0  4974.0      60.0    59.07        7362.0  8290.0     100.0   

Tool             STC-seq                            Kourami_ambiguous          \
Metric  accuracy   score   count call_rate accuracy             score   count   
A          81.00   843.0  1162.0     70.08    50.84            1423.0  1600.0   
B          92.94   769.0  1180.0     71.17    46.38            1297.0  1432.0   
C          92.76   738.0  1126.0     67.91    44.51            1362.0  1500.0   
DRB1       88.42  1033.0  1280.0     77.20    62.30            1591.0  1648.0   
DQB1       88.90   783.0   960.0     57.90    47.23            1228.0  1388.0   
HLA-I      88.90  2350.0  3468.0     69.72    47.25            4082.0  4532.0   
HLA-II     88.66  1816.0  2240.0     67.55    54.76            2819.0  3036.0   
Evaxion    88.78  3383.0  4748.0     71.59    51.01            5673.0  6180.0   
Total      88.81  4166.0  5708.0     68.85    50.25            6901.0  7568.0   

Tool                       ensemble_all                             \
Metric  call_rate accuracy        score   count call_rate accuracy   
A           96.50    85.83       1581.0  1658.0     100.0    95.36   
B           86.37    78.23       1607.0  1658.0     100.0    96.92   
C           90.47    82.15       1600.0  1658.0     100.0    96.50   
DRB1        99.40    95.96       1605.0  1658.0     100.0    96.80   
DQB1        83.72    74.07       1443.0  1658.0     100.0    87.03   
HLA-I       91.11    82.07       4788.0  4974.0     100.0    96.26   
HLA-II      91.56    85.01       3048.0  3316.0     100.0    91.92   
Evaxion     93.18    85.54       6393.0  6632.0     100.0    96.40   
Total       91.29    83.24       7836.0  8290.0     100.0    94.52   

Tool    ensemble_graph                            Total  
Metric           score   count call_rate accuracy count  
A               1489.0  1658.0     100.0    89.81  1658  
B               1581.0  1658.0     100.0    95.36  1658  
C               1565.0  1658.0     100.0    94.39  1658  
DRB1            1605.0  1658.0     100.0    96.80  1658  
DQB1            1448.0  1658.0     100.0    87.33  1658  
HLA-I           4635.0  4974.0     100.0    93.18  4974  
HLA-II          3053.0  3316.0     100.0    92.07  3316  
Evaxion         6240.0  6632.0     100.0    94.09  6632  
Total           7688.0  8290

As seen above, the ambiguous typing of Kourami doesn't give a great increase in performance, and it is therefore disregarded in futher analysis

In [30]:
results_df = results_df.drop('Kourami_ambiguous', axis = 1)

C:\Users\nikol\Anaconda3\lib\site-packages\pandas\core\generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


# Add metadata to error dataframe

In [31]:
#Join the error dataframe with information of continent and ethnicity
errors_with_metadata_hla_I_df_1 = error_dataframe_hla_I.join(gs_2018_raw_df[['Region', 'sbgroup']])
errors_with_metadata_hla_I_df_1.rename(columns={'sbgroup' : 'Population'}, inplace=True)

#The same for HLA-II
errors_with_metadata_hla_II_df_1 = error_dataframe_hla_II.join(gs_2018_raw_df[['Region', 'sbgroup']])
errors_with_metadata_hla_II_df_1.rename(columns={'sbgroup' : 'Population'}, inplace=True)

In [32]:
#Datapath for metadata excelsheet:
metadata_path = gold_standard_path + '1000G_sample_info.xlsx'

#Load data for gender and translation of Population abbreviations
metadata_1_df = pd.read_excel(metadata_path, sheet_name='Sample Info')[['Sample', 'Population', 'Population Description', 'Gender']]

#load data for sequencing center:
metadata_2_df = pd.read_excel(metadata_path, sheet_name='Final Phase Sequence Data')[['Unnamed: 0', 'Exome']]

#Reset header in metadata_2
new_header = metadata_2_df.iloc[0] #grab the first row for the header
metadata_2_df = metadata_2_df[1:] #take the data less the header row
metadata_2_df.columns = new_header #set the header row as the df header

#Set "Sample" as inde in both metadata dataframes:
metadata_1_df.set_index('Sample', inplace=True)
metadata_2_df.set_index('Sample', inplace=True)

In [33]:
#Join the relevant info to the error dataframe:
errors_with_metadata_hla_I_df_2 = errors_with_metadata_hla_I_df_1.join(metadata_1_df[['Population Description', 'Gender']])

errors_metadata_hla_I_df = errors_with_metadata_hla_I_df_2.join(metadata_2_df[['Center']])

#Do the same for HLA-II
errors_with_metadata_hla_II_df_2 = errors_with_metadata_hla_II_df_1.join(metadata_1_df[['Population Description', 'Gender']])

errors_metadata_hla_II_df = errors_with_metadata_hla_II_df_2.join(metadata_2_df[['Center']])

### Add a column with the sum of errors for all but STC-seq

In [34]:
errors_metadata_hla_I_df['mean_error_graph'] = errors_metadata_hla_I_df.apply(lambda row: (row['Kourami'] + row['HLA-LA'] + row['Hisatgenotype']) / 3  , axis = 1) 

errors_metadata_hla_II_df['mean_error_graph'] = errors_metadata_hla_II_df.apply(lambda row: (row['Kourami'] + row['HLA-LA'] + row['Hisatgenotype']) / 3  , axis = 1) 


# Save all dataframes with results as pickle objects


In [35]:
#Overall result dataframe:

if (subsampling == False) and (high_coverage_analysis == False):
    results_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_results_df.pkl")

    errors_metadata_hla_I_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_errors_metadata_hla_I_df.pkl")

    errors_metadata_hla_II_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_errors_metadata_hla_II_df.pkl")

#For subsampling:
elif high_coverage_analysis == False:

    results_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_results_df_" + subsampling + "X.pkl")
    errors_metadata_hla_I_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_errors_metadata_hla_I_df_" + subsampling + "X.pkl")
    errors_metadata_hla_II_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_errors_metadata_hla_II_df_" + subsampling + "X.pkl")

else:    
    results_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_above200X_results_df.pkl")
    errors_metadata_hla_I_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_above200X_errors_metadata_hla_I_df.pkl")
    errors_metadata_hla_II_df.to_pickle(main_folder_path + "result_dataframes\\" + resolution + "_above200X_errors_metadata_hla_II_df.pkl")
